In [1]:
import os
import json
import cv2
from pathlib import Path
from tqdm import tqdm

VIDEOS_ROOT = 'SoccerNet/videos'           
OUTPUT_DIR = 'dataset-camera'          
LABELS_FILE = 'Labels-cameras.json'



def extract_frame(video_path, time):
    """
    Extrae un frame del video en el tiempo especificado.
    :param video_path: Ruta al video.
    :param time: Tiempo en segundos para extraer el frame.
    :return: Lista con el frame extraído o None si falla.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return None
    # Calcular el frame correspondiente al tiempo
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_number = int(fps * time)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    success, frame = cap.read()
    cap.release()
    if not success:
        return None
    return frame

def process_match(partido_path):
    json_path = os.path.join(partido_path, LABELS_FILE)
    if not os.path.exists(json_path):
        print(f"⚠️ No se encontró JSON en {partido_path}")
        return

    with open(json_path, 'r') as f:
        data = json.load(f)['annotations']

    # Crear índice de videos
    video_paths = {
        "1": os.path.join(partido_path, "1_224p.mkv"),
        "2": os.path.join(partido_path, "2_224p.mkv")
    }
    #si falta el video de la mitad 1 o 2, se omite
    if not os.path.exists(video_paths["1"]) and not os.path.exists(video_paths["2"]):
        return False
    for entry in tqdm(data, desc=f"Procesando {Path(partido_path).name}"):
        game_time = entry['gameTime'].split(' - ')
        label = entry['label']
        half =game_time[0].strip()  # "1" o "2"
        time = game_time[1].strip()  # "00:00:00" o similar
        #pasamos el tiempo a segundos
        time_parts = time.split(':')
        if len(time_parts) == 3:
            seconds = int(time_parts[0]) * 3600 + int(time_parts[1]) * 60 + int(time_parts[2])
        elif len(time_parts) == 2:
            seconds = int(time_parts[0]) * 60 + int(time_parts[1])
        else:
            seconds = 0
        time = seconds
        video_path = video_paths.get(half)

        class_name = label.replace(" ", "_").lower().strip()

        class_dir = os.path.join(OUTPUT_DIR, class_name)
        os.makedirs(class_dir, exist_ok=True)

        # Extraer y guardar el frame
        frame = extract_frame(video_path, time)
        if frame is not None:
            frame_filename = f"{Path(partido_path).name}_{half}_{time}.jpg"
            frame_path = os.path.join(class_dir, frame_filename)
            cv2.imwrite(frame_path, frame)
    return True

index = 0
for liga in os.listdir(VIDEOS_ROOT):
    liga_path = os.path.join(VIDEOS_ROOT, liga)
    for temporada in os.listdir(liga_path):
        temporada_path = os.path.join(liga_path, temporada)
        for partido in os.listdir(temporada_path):
            # Usar la ruta correcta para partido_path incluyendo temporada_path
            partido_path = os.path.join(temporada_path, partido)
            if os.path.isdir(partido_path):
                r = process_match(partido_path)
                if r:
                    index += 1
print(f"Procesamiento completado. Se procesaron {index} partidos.")


Procesando 2015-02-22 - 19-15 Southampton 0 - 2 Liverpool: 100%|██████████| 516/516 [00:10<00:00, 47.74it/s]
Procesando 2015-02-21 - 18-00 Chelsea 1 - 1 Burnley: 100%|██████████| 413/413 [00:08<00:00, 47.13it/s]
Procesando 2015-02-21 - 18-00 Swansea 2 - 1 Manchester United: 100%|██████████| 579/579 [00:12<00:00, 48.17it/s]
Procesando 2015-02-21 - 18-00 Crystal Palace 1 - 2 Arsenal: 100%|██████████| 745/745 [00:16<00:00, 45.89it/s]
Procesando 2015-10-31 - 15-45 Chelsea 1 - 3 Liverpool: 100%|██████████| 145/145 [00:03<00:00, 43.35it/s]
Procesando 2015-09-12 - 14-45 Everton 3 - 1 Chelsea: 100%|██████████| 121/121 [00:02<00:00, 42.69it/s]
Procesando 2016-02-14 - 19-15 Manchester City 1 - 2 Tottenham: 100%|██████████| 508/508 [00:10<00:00, 46.52it/s]
Procesando 2015-08-29 - 17-00 Chelsea 1 - 2 Crystal Palace: 100%|██████████| 117/117 [00:02<00:00, 43.68it/s]
Procesando 2016-01-03 - 16-30 Crystal Palace 0 - 3 Chelsea: 100%|██████████| 456/456 [00:10<00:00, 43.92it/s]
Procesando 2016-02-07 - 

Procesamiento completado. Se procesaron 272 partidos.
